<a href="https://colab.research.google.com/github/marcocucei/TI_I_Practica4/blob/main/TI_I_Practica4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica 4 - Dashboard Random Walk + Path Length/MSD

**Nombre:** Marco Antonio Hernández Preciado  
**e-mail:** marco.hpreciado@alumnos.udg.mx

# Bibliotecas

In [2]:
import panel as pn
import panel.widgets as pnw

pn.extension('plotly')

import pandas as pd
import numpy as np

import plotly.graph_objects as go

import math

from scipy.stats import wrapcauchy
from scipy.stats import levy_stable

##CLASSES

In [4]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y

    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)

    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)

    # rotate vector
    def rotated(self, angle):
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

## Funciones para generar las trayectorias

In [5]:
###############################################################################################
# Brownian Motion (BM) Trajectory
###############################################################################################
def bm_2d(n_steps=1000, speed=6, s_x_pos=0, s_y_pos=0):
    """
    Arguments:
        n_steps:
        speed:
        s_pos:
    Returns:
        BM_2d_df:
    """
    # Init velocity vector
    velocity = Vec2d(speed,0)

    BM_2d_df = pd.DataFrame(columns = ['x_pos','y_pos'])
    temp_df = pd.DataFrame([{'x_pos': s_x_pos, 'y_pos': s_y_pos}])
    BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)


    for i in range(n_steps-1):
        turn_angle = np.random.uniform(low=-np.pi, high=np.pi)
        velocity = velocity.rotated(turn_angle)

        temp_df = pd.DataFrame([{'x_pos': BM_2d_df.x_pos[i]+velocity.x, 'y_pos': BM_2d_df.y_pos[i]+velocity.y}])
        BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

    # return dataframe
    return BM_2d_df

In [29]:
###############################################################################################
# Correlated Random Walk (CRW) Trajectory
###############################################################################################

def crw_2d(n_steps = 1000, speed = 6, s_x_pos=0, s_y_pos=0, cauchy_dist = 0.7):

  #Init velocity vector
  velocity = Vec2d(speed, 0)

  CRW_2d_df =  pd.DataFrame(columns = ['x_pos', 'y_pos'])
  temp_df = pd.DataFrame([{'x_pos': s_x_pos, 'y_pos': s_y_pos}])
  CRW_2d_df = pd.concat([CRW_2d_df, temp_df], ignore_index = True)

  bm_turning_angle = wrapcauchy.rvs(c=cauchy_dist, loc = 0, size = n_steps - 1)

  for i in range(1, n_steps):
    angulo_giro = bm_turning_angle[i-1]
    velocity = velocity.rotated(angulo_giro)

    temp_df = pd.DataFrame([{"x_pos": CRW_2d_df.x_pos[i-1] + velocity.x,
                             "y_pos": CRW_2d_df.y_pos[i-1] + velocity.y}])
    CRW_2d_df = pd.concat([CRW_2d_df, temp_df], ignore_index=True)

  return CRW_2d_df

In [41]:
###############################################################################################
# Levy Flight (LF) Trajectory
###############################################################################################

def levy_2d(n_steps = 1000, speed = 6, s_x_pos=0, s_y_pos=0, cauchy_dist=0.4, exp_lv = 1 ):

  velocity = Vec2d(speed, 0)
  std_steps = 3
  levy_flight_df = pd.DataFrame(columns=["x_pos", "y_pos"])
  temp_df = pd.DataFrame([{"x_pos": s_x_pos, "y_pos": s_y_pos}])
  levy_flight_df = pd.concat([levy_flight_df, temp_df], ignore_index=True)

  i=1
  while i < n_steps:
    angulo_giro = wrapcauchy.rvs(c=cauchy_dist, loc=0)
    velocity = velocity.rotated(angulo_giro)
    num_steps = abs(math.floor(levy_stable.rvs(alpha=exp_lv, beta=1, loc=std_steps)))
    if (i + num_steps > n_steps):
      num_steps = n_steps - 1

    contador_paso = 0
    while contador_paso < num_steps:
      temp_df = pd.DataFrame([{"x_pos": levy_flight_df.x_pos[i-1] + velocity.x,
                               "y_pos": levy_flight_df.y_pos[i-1] + velocity.y}])
      levy_flight_df = pd.concat([levy_flight_df, temp_df], ignore_index=True)
      contador_paso += 1
      i += 1

  return levy_flight_df

#Parametros

In [133]:
n_steps = pnw.IntSlider(name='Number of steps',value=20,step=1,start=0,end=1000, width = 300)
speed = pnw.IntSlider(name='Speed', value=6,step=1,start=0,end=10, width = 300)
s_x_pos = pnw.FloatInput(name='Starting pos_x',value=0,step=1,start=0,end=10000, width = 140)
s_y_pos = pnw.FloatInput(name='Starting pos_y',value=0,step=1,start=0,end=10000, width = 140)
cauchy_dist =pnw.FloatInput(name='Cauchy coefficient',value=0.6, step=0.1, start=0.1, end=1, width = 140)
exp_lv = pnw.FloatInput(name='Levy exponent',value=0.4,step=0.1,start=0.1,end=1, width = 140)

## Elegir numero de pasos y mostrar gráfico

In [104]:
# Definir el widget para ingresar el numero de pasos
#n_steps = pnw.IntInput(name='Number of steps:', value=20, step=10, start=1, end=1000)

@pn.depends(n_steps, speed, s_x_pos, s_y_pos)
def plot_traj_bm(n_steps, speed, s_x_pos, s_y_pos):
  random_walker_df=bm_2d(n_steps, speed, s_x_pos, s_y_pos)

  fig_traj_rw = go.Figure()

  fig_traj_rw.add_trace(
      go.Scatter3d(x=random_walker_df.x_pos,
                 y=random_walker_df.y_pos,
                 z = random_walker_df.index,
                 name = f'steps = {n_steps}',
                 mode = 'lines',
                 showlegend = True)
  )

  return fig_traj_rw

#pn.Column(n_steps, plot_traj_bm)

In [107]:
# Definir el widget para ingresar el numero de pasos
#n_steps = pnw.IntInput(name='Number of steps:', value=20, step=10, start=1, end=1000)

@pn.depends(n_steps, speed, s_x_pos, s_y_pos, cauchy_dist)
def plot_traj_crw(n_steps, speed, s_x_pos, s_y_pos, cauchy_dist):
  crw_df=crw_2d(n_steps, speed, s_x_pos, s_y_pos, cauchy_dist)

  fig_traj_crw = go.Figure()

  fig_traj_crw.add_trace(
      go.Scatter3d(x = crw_df.x_pos,
                   y = crw_df.y_pos,
                   z = crw_df.index,
                   name = f'steps = {n_steps}',
                   mode = 'lines',
                   showlegend = True)
  )

  return fig_traj_crw

#pn.Column(n_steps, plot_traj_crw)

In [110]:
# Definir el widget para ingresar el numero de pasos
#n_steps = pnw.IntInput(name='Number of steps:', value=20, step=10, start=1, end=1000)

@pn.depends(n_steps, speed, s_x_pos, s_y_pos, cauchy_dist, exp_lv)
def plot_traj_levy(n_steps, speed, s_x_pos, s_y_pos, cauchy_dist, exp_lv):
  levy_df=levy_2d(n_steps, speed, s_x_pos, s_y_pos, cauchy_dist, exp_lv)

  fig_traj_levy = go.Figure()

  fig_traj_levy.add_trace(
      go.Scatter3d(x=levy_df.x_pos,
                   y=levy_df.y_pos,
                   z = levy_df.index,
                   mode = 'lines',
                   name = f'steps = {n_steps}',
                   showlegend = True)
  )

  return fig_traj_levy

#pn.Column(n_steps, plot_traj_levy)

## Funciones para calcular las métricas

### Path length

In [116]:
#Path length Brownian  Motion

@pn.depends(n_steps, speed, s_x_pos, s_y_pos)
def metrica_bm(n_steps, speed, s_x_pos, s_y_pos):
  fig_met_bm = go.Figure()
  random_walker_df = bm_2d(n_steps, speed, s_x_pos, s_y_pos)
  metrica_bm_df = ([np.linalg.norm(random_walker_df.iloc[i-1]-random_walker_df.iloc[i]) for i in range(1, random_walker_df.shape[0])])

  plot_bm = np.cumsum(metrica_bm_df)
  fig_met_bm.add_trace(go.Scatter(
      x = np.arange(len(plot_bm)),
      y = plot_bm,
      mode = 'lines',
      name = 'Path Length',
      showlegend = True
  ))

  fig_met_bm.update_layout(title_text = 'Path Length')

  return fig_met_bm


In [113]:
#pn.Column(metrica_bm)

In [117]:
# Path Length CRW

@pn.depends(n_steps, speed, s_x_pos, s_y_pos, cauchy_dist)
def metrica_crw(n_steps, speed, s_x_pos, s_y_pos, cauchy_dist):
  fig_met_crw = go.Figure()
  crw_df = crw_2d(n_steps, speed, s_x_pos, s_y_pos, cauchy_dist)
  metrica_crw_df = ([np.linalg.norm(crw_df.iloc[i-1]-crw_df.iloc[i]) for i in range(1, crw_df.shape[0])])

  plot_crw = np.cumsum(metrica_crw_df)
  fig_met_crw.add_trace(go.Scatter(
      x = np.arange(len(plot_crw)),
      y = plot_crw,
      mode = 'lines',
      name = 'Path Length',
      showlegend = True
  ))

  fig_met_crw.update_layout(title_text = 'Path Length')

  return fig_met_crw

In [115]:
#pn.Column(metrica_crw)

In [118]:
# Path Length Levy

@pn.depends(n_steps, speed, s_x_pos, s_y_pos, cauchy_dist, exp_lv)
def metrica_levy(n_steps, speed, s_x_pos, s_y_pos, cauchy_dist, exp_lv):
  fig_met_levy = go.Figure()
  levy_df = levy_2d(n_steps, speed, s_x_pos, s_y_pos, cauchy_dist, exp_lv)
  metrica_levy_df = ([np.linalg.norm(levy_df.iloc[i-1]-levy_df.iloc[i]) for i in range(1, levy_df.shape[0])])

  plot_levy = np.cumsum(metrica_levy_df)
  fig_met_levy.add_trace(go.Scatter(
      x = np.arange(len(plot_levy)),
      y = plot_levy,
      mode = 'lines',
      name = 'Path Length',
      showlegend = True
  ))

  fig_met_levy.update_layout(title_text = 'Path Length')

  return fig_met_levy

In [120]:
#pn.Column(metrica_levy)

###Mean Squared Displacement

In [121]:
# Mean Squared Displacement BM

@pn.depends(n_steps, speed, s_x_pos, s_y_pos)
def graf_msd_bm(n_steps, speed, s_x_pos, s_y_pos):
  msd_brownian = np.empty(shape=(0))
  fig_msd_bm = go.Figure()
  random_walker_df = bm_2d(n_steps, speed, s_x_pos, s_y_pos)

  for tau in range(1, random_walker_df.shape[0]):
    metrica_bm = ([(np.linalg.norm(random_walker_df.iloc[i-tau]-random_walker_df.iloc[i]))**2
                   for i in range (tau, random_walker_df.shape[0],1)])
    d = np.mean(metrica_bm)
    msd_brownian = np.append(msd_brownian, d)
    msd_brownian_df = pd.DataFrame()
    msd_brownian_df['msd_brownian'] = msd_brownian

  fig_msd_bm.add_trace(go.Scatter(
      x = np.arange(len(msd_brownian_df.index)),
      y = msd_brownian_df.msd_brownian,
      mode = 'lines',
      name = 'MSD BM',
      showlegend = True
    ))

  return fig_msd_bm


In [123]:
#pn.Column(graf_msd_bm)

In [124]:
# Mean Squared Displacement CRW

@pn.depends(n_steps, speed, s_x_pos, s_y_pos, cauchy_dist)
def graf_msd_crw(n_steps, speed, s_x_pos, s_y_pos, cauchy_dist):
  msd_crw = np.empty(shape = (0))
  fig_msd_crw = go.Figure()
  crw_df = crw_2d(n_steps, speed, s_x_pos, s_y_pos, cauchy_dist)

  for tau in range(1, crw_df.shape[0]):
    metrica_crw = ([(np.linalg.norm(crw_df.iloc[i-tau]-crw_df.iloc[i]))**2
                   for i in range(tau, crw_df.shape[0],1)])
    d = np.mean(metrica_crw)
    msd_crw = np.append(msd_crw, d)
    msd_crw_df = pd.DataFrame()
    msd_crw_df['msd_crw'] = msd_crw

  fig_msd_crw.add_trace(go.Scatter(
      x = np.arange(len(msd_crw_df.index)),
      y = msd_crw_df.msd_crw,
      mode = 'lines',
      name = 'Msd CRW',
      showlegend = True
  ))

  return fig_msd_crw

In [126]:
#pn.Column(graf_msd_crw)

In [127]:
# Mean Squared Displacement CRW

@pn.depends(n_steps, speed, s_x_pos, s_y_pos, cauchy_dist, exp_lv)

def graf_msd_levy(n_steps, speed, s_x_pos, s_y_pos, cauchy_dist, exp_lv):
  msd_levy = np.empty(shape=(0))
  fig_msd_levy = go.Figure()
  levy_df = levy_2d(n_steps, speed, s_x_pos, s_y_pos, cauchy_dist, exp_lv)

  for tau in range(1, levy_df.shape[0]):
    metrica_levy = ([(np.linalg.norm(levy_df.iloc[i-tau]-levy_df.iloc[i]))**2
                    for i in range(tau, levy_df.shape[0],1)])
    d = np.mean(metrica_levy)
    msd_levy = np.append(msd_levy, d)
    msd_levy_df = pd.DataFrame()
    msd_levy_df['msd_levy'] = msd_levy

  fig_msd_levy.add_trace(go.Scatter(
      x = np.arange(len(msd_levy_df.index)),
      y = msd_levy_df.msd_levy,
      mode = 'lines',
      name = 'Msd Levy',
      showlegend=True
))

  return fig_msd_levy

In [129]:
#pn.Column(graf_msd_levy)

# DASHBOARD

In [87]:
#Definición de paneles

#Grafico en 3d
trayectoria_3d_pane = pn.pane.Plotly()
#Grafico metrica
metrica_pane = pn.pane.Plotly()


#Definición de widgets

#Elegir Trayectoria
trayectoria_selector = pnw.RadioButtonGroup(options=['BM', 'CRW', ' LF'], value = 'BM', name = 'Tipo de trayectoria', width = 300)
#Elegir Metrica
metrica_selector = pnw.Select(options=['PL', 'MSD'], name = 'Tipo de metrica', value = 'PL', width= 300)

In [136]:
#Actualizar tipo de trayectoria

def actualizar_tipo_trayectoria(event):
  tipo_trayectoria = trayectoria_selector.value

  if tipo_trayectoria == 'BM':
    #trayectoria_df = bm_2d(n_steps, speed, s_x_pos, s_y_pos)

    cauchy_dist.visible = False
    exp_lv.visible = False
  elif tipo_trayectoria == 'CRW':
    exp_lv.visible = False

In [141]:
dashboard = pn.Row(
    pn.Column(
        trayectoria_selector,
        n_steps,
        pn.Row(
            s_x_pos,
            s_y_pos
        ),
        speed,
        metrica_selector,
        pn.Row(
            cauchy_dist,
            exp_lv
        )
    ),
    #trayectoria_3d_pane,
    #metrica_pane
    plot_traj_bm

)

dashboard.servable()

Row
    [0] Column
        [0] RadioButtonGroup(name='Tipo de trayectoria', options=['BM', 'CRW', ' LF'], value='BM', width=300)
        [1] IntSlider(end=1000, name='Number of steps', value=86, width=300)
        [2] Row
            [0] FloatInput(end=10000, name='Starting pos_x', start=0, step=1, width=140)
            [1] FloatInput(end=10000, name='Starting pos_y', start=0, step=1, width=140)
        [3] IntSlider(end=10, name='Speed', value=6, width=300)
        [4] Select(name='Tipo de metrica', options=['PL', 'MSD'], value='PL')
        [5] Row
            [0] FloatInput(end=1, name='Cauchy coefficient', start=0.1, value=0.6, width=140)
            [1] FloatInput(end=1, name='Levy exponent', start=0.1, value=0.4, width=140)
    [1] ParamFunction(function, _pane=Plotly, defer_load=False)